In [308]:
import numpy as np
from scipy import constants
import scipy.integrate
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('text', usetex=True)
from scipy.special import factorial
Ry_in_erg = constants.value('Rydberg constant times hc in J') * 1.e7 #in ergs
h = constants.value('Planck constant') * 1e7 #in ergs s
c = constants.c * 1.e2 # in cm/s
e_e = 4.80320451e-10 #in statcoulomb
m_e = constants.electron_mass * 1e3 # in g

In [320]:
def gaunt_bound_free(n, l, E, Z, Eph):
    E0 = np.copy(E)
    if E0 < Eph:
        E0 = Eph
        
    sigmaA = sigma_plus(n, l, E0, Z)
    sigmaB = sigma_minus(n, l, E0, Z)
    sigmaK = sigma_K(n, E0, Z)
    return (sigmaA + sigmaB) / sigmaK


In [321]:
def sigma_K(n, E, Z, nu=None):
    eta = (Z**2 * Ry_in_erg / E)**.5
    rho = eta / n
    nu_A = 1
    if nu:
        nu_A /= nu
    return nu_A * 2.**4 / 3. / 3.**.5 *(e_e**2 / m_e / c) / n * (rho**2 / (1+rho**2))**2

In [322]:
def G_l(l, m, eta, rho):
    """G matrix element"""
    gl = 0
    for i in range(2*m + 1):
        gl += b_s(i, l, m, eta, rho) * rho**i
        
    return gl
def b_s(s, l, m, eta, rho):
    """polynomial solution to matrix element G_l"""
    if s == 0:
        return 1
    if s == 1:
        return 2 * m * eta / l
    
    bsminus1 = b_s(s-1, l, m, eta, rho)
    bsminus2 = b_s(s-2, l, m, eta, rho)
    bs = -(1/(s*(s + 2*l-1))) * (4*eta * (s - 1 - m)*bsminus1 + (2 * m + 2. - s)* (2*m + 2*l + 1 - s) * bsminus2)
    return bs

In [323]:
def sigma_minus(n, l, E, Z, nu=None):
    if l == 0:
        return 0
    eta = (Z**2 * Ry_in_erg / E)**.5
    rho = eta / n
    GlA = G_l(l, -(l+1-n), eta, rho)
    GlB = G_l(l, -(l-1-n), eta, rho)

    nu_A = 1
    if nu:
        nu_A /= nu
    prefactor = 2**(4*l)/3*np.pi * e_e**2/m_e/c
    
    A = 1
    for l_i in range(1, l):
        A*= (l_i**2 + eta**2)
    B = l**2 * factorial(n+l) / factorial(2*l +1) /factorial(2*l-1)/factorial(n-l-1)
    C = np.exp(-4*eta * np.arctan2(1,rho)) / (1-np.exp(-2*np.pi*eta)) 
    D = rho**(2*l+2)/(1+rho**2)**(2*n-2)
    E = (GlA - (1+rho**2)**(-2)*GlB)**2
    return nu_A*prefactor*A *B*C*D*E

def sigma_plus(n, l, E, Z, nu=None):
    eta = (Z**2 * Ry_in_erg / E)**.5
    rho = eta / n
    GlA = G_l(l+1, -(l+1-n), eta, rho)
    GlB = G_l(l+1, -(l-n), eta, rho)

    nu_A = 1
    if nu:
        nu_A /= nu
    prefactor = 2**(4*l + 6)/3*np.pi * e_e**2/m_e/c
    
    A = 1 + eta**2
    for l_i in range(2, l+2):
        A*= (l_i**2 + eta**2)
    B = (l+1)**2 * factorial(n+l) / (2*l+1)/factorial(2*l +1) /factorial(2*l+2)/factorial(n-l-1)/factorial((l+1)**2+eta**2)**2
    
    C = np.exp(-4*eta * np.arctan2(1,rho)) / (1-np.exp(-2*np.pi*eta)) 
    D = rho**(2*l+4)*eta**2/(1+rho**2)**(2*n)
    E = ((l+1-n)*GlA + (l+1+n)/(1+rho**2)*GlB)**2
    return nu_A*prefactor*A *B*C*D*E    

In [334]:
electronenergy = [.1e13,.1111e12,.1e11,.1111e10,.1e9,.1111e8,.1e7,.1111e6,.4e5,.2041e5, .1e5,.4444e4,.25e4,.1111e4,.4e3,.2041e3,.1e3,.4444e2,.25e2,.16e2,.1111e2,.625e1,.4e1,.2778e1,.2041e1,.1562e1,.1235e1,.1e1,.6944,.4444,.25,.11111,.4e-1, .2041e-1,.1e-1,.4444e-2,.25e-2,.1111e-2]
photonenergy = [.1e13,.1111e12,.1e11,.1111e10,.1e9,.1111e8,.1e7,.1111e6,.4e5,.2041e5, .1e5,.4445e4,.2501e4,.1112e4,.401e3,.2051e3,.101e3,.4544e2,.26e2,.17e2,.1211e2,.725e1,.5e1,.3778e1,.3041e1,.2562e1,.2235e1,.2e1,1.6944,1.4444,1.25,1.1111,1.04, 1.02,1.01,1.004,1.002,1.001]

ys = []
for i in range(len(electronenergy)):
    ys.append( gaunt_bound_free(1,0,electronenergy[i]* Ry_in_erg,1, photonenergy[i] * Ry_in_erg))

In [335]:
plt.semilogx(electronenergy, ys)

In [331]:
list(zip(test,ys))

[(1000000000000.0, 6.9281814646873407e-06),
 (111100000000.0, 2.0785453090713404e-05),
 (10000000000.0, 6.9279855795087938e-05),
 (1111000000.0, 0.00020783690039304518),
 (100000000.0, 0.00069260271867043323),
 (11110000.0, 0.0020766071927210247),
 (1000000.0, 0.0069064891232228287),
 (111100.0, 0.020591124531417159),
 (40000.0, 0.034103267159912062),
 (20410.0, 0.047446295347699327),
 (10000.0, 0.067156156547138371),
 (4444.0, 0.099189457546235732),
 (2500.0, 0.13023188814868392),
 (1111.0, 0.18951025140825525),
 (400.0, 0.29759618568150009),
 (204.1, 0.39324453311592755),
 (100.0, 0.51690178369156337),
 (44.44, 0.68089597570722127),
 (25.0, 0.80584840478086706),
 (16.0, 0.90315820137464375),
 (11.11, 0.98051820748880913),
 (6.25, 1.0940239783694701),
 (4.0, 1.1698942157674908),
 (2.778, 1.2189081465628238),
 (2.041, 1.2473324761496016),
 (1.562, 1.2599646718618407),
 (12.35, 1.2609443966292753),
 (1.0, 1.2538490032718501),
 (0.6944, 1.2262429013557297),
 (0.4444, 1.1753733270018245),